In [1]:
import numpy as np

In [4]:
class RNN:
    def __init__(self, Wx, Wh, b) :
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev) :
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tahn(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next) :
        Wx, Wh, b = self.params # 매개변수 2개 지정
        x, h_prev, h_next = self.cache # 중간 데이터 저장
        
        dt = dh_next * (1-h_next ** 2) # tanh 함수 미분 값 * h_next 미분 값
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev
    

In [9]:
class TimeRNN :
    def __init__(self, Wx, Wh, b, stateful=False) :
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None # 다수의 RNN 계층을 리스트로 저장
        
        self.h, self.dh = None, None
        self.stateful = stateful
    
    def set_state(self, h) :
        self.h = h
    
    def reset_state(self) :
        self.h = None
    
    def forward(self, xs) :
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 미니배치 크기 N, T개 분량의 시계열, 입력벡터의 차원 수 D
        D, H = Wx.shape # 매개변수 모양
        
        self.layers = []
        hs = np.empty((N,T,H), dtype = 'f') # 출력값을 담을 그릇 준비
        
        if not self.stateful or self.h is None :
            self.h = np.zeros((N,H), dtype = 'f') # stateful도 false라면, self.h가 없다면 영행렬 호출
        
        for t in range(T):
            layer = RNN(*self.params) # t회 RNN 계층 생성 반복
            self.h = layer.forward(xs[:,t,:], self.h) # 처음 self.h는 영행렬이지만 xs가 입력되고 RNN을 거쳐 출력된 값으로 업데이트 됨
            hs[:,t,:] = self.h # t 인덱스 별로 hs 값 입력
            self.layers.append(layer)
            
        return hs # t개 있음
    
    def backward(self, dhs) :
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 미니배치 크기 N, T개 분량의 시계열, 입력벡터의 차원 수 D
        D, H = Wx.shape    
        
        dxs = np.empty((N,T,D), dtype= 'f')
        dh = 0 
        grads = [0,0,0]
        for t in reversed(range(T)) :
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:,t,:] + dh) # 합산된 기울기
            dxs[:,t,:] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        for i, grad in enumerate(grads) :
            self.grads[i][...] = grad
        self.dh = dh
        return dxs